# XL-BOMD
## see https://aip.scitation.org/doi/full/10.1063/1.3148075 for details.

In [1]:
import sys
sys.path.insert(1, "/home/maxim/Projects/git2/PYSEQM_dev/")

import torch
from seqm.seqm_functions.constants import Constants
from seqm.Molecule import Molecule
from seqm.MolecularDynamics import XL_BOMD

torch.set_default_dtype(torch.float64)
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

/home/maxim/anaconda3/envs/hipnn_1/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Decorating your function! <function KSA_XL_BOMD.one_step at 0x7ffadb6dcca0>


In [7]:
%%time
torch.manual_seed(0)

species = torch.as_tensor([[8,6,1,1],[5,1,1,1]],dtype=torch.int64, device=device)[:]
coordinates = torch.tensor([
                  [
                   [ 0.00,  0.0,  0.0],
                   [ 1.22,  0.0,  0.0],
                   [ 1.82,  0.94, 0.0],
                   [ 1.82, -0.94, 0.0]
                  ],
                  [
                   [ 0.00,  0.00,  0.00],
                   [ 1.20,  0.00,  0.00],
                   [-0.60,  1.03,  0.00],
                   [-0.60, -1.03,  0.00]
                  ]
                 ], device=device)[:]

const = Constants().to(device)

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2,0.0], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True
                   }

molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

# Set k for the dissipative electronic force term. See https://aip.scitation.org/doi/full/10.1063/1.3148075 for details.
output={'molid':[0, 1], 'thermo':1, 'dump':1, 'prefix':'Outputs/4_XL-BOMD'}
xl_bomd_params={'k':6}

md =  XL_BOMD(xl_bomd_params=xl_bomd_params,
              seqm_parameters=seqm_parameters, Temp=400.0, timestep=0.4, output=output).to(device)
md.initialize_velocity(molecule )
#remove center of mass velocity
# Info_log flag generates .txt wtih additional info for every step (orbital energies, dipole)
_ = md.run(molecule, 20, remove_com=[True, 1], Info_log=True)

Initialize velocities: zero_com
Doing initialization
Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   279.98   1.447585e-01 -1.350261e+00 -1.205502e+00 ||    377.92   1.953992e-01 1.466852e+00 1.662251e+00 || 
     2   240.90   1.245575e-01 -1.330494e+00 -1.205936e+00 ||    338.35   1.749386e-01 1.487137e+00 1.662075e+00 || 
     3   191.92   9.922878e-02 -1.305360e+00 -1.206131e+00 ||    328.43   1.698119e-01 1.492034e+00 1.661846e+00 || 
     4   139.86   7.231308e-02 -1.277697e+00 -1.205384e+00 ||    346.89   1.793582e-01 1.482438e+00 1.661796e+00 || 
     5    90.98   4.704031e-02 -1.251801e+00 -1.204761e+00 ||    388.52   2.008808e-01 1.460645e+00 1.661526e+00 || 
     6    51.10   2.641887e-02 -1.230973e+00 -1.204554e+00 ||    445.38   2.302809e-01 1.430938e+00 1.661219e+00 || 
     7    25.42   1.314202e-02 -1.217666e+00 -1.204524e+00 ||    507.92   2.626136e-01 1.398362e+00 1.660976e+00 || 
     8    17.41   9.001535e-03 -1.213564e+00 -1.204563e+00 ||    566.07  

## Damping (Langevin)

In [8]:
%%time
torch.manual_seed(0)

species = torch.as_tensor([[8,6,1,1],[5,1,1,1]],dtype=torch.int64, device=device)[:]
coordinates = torch.tensor([
                  [
                   [ 0.00,  0.0,  0.0],
                   [ 1.22,  0.0,  0.0],
                   [ 1.82,  0.94, 0.0],
                   [ 1.82, -0.94, 0.0]
                  ],
                  [
                   [ 0.00,  0.00,  0.00],
                   [ 1.20,  0.00,  0.00],
                   [-0.60,  1.03,  0.00],
                   [-0.60, -1.03,  0.00]
                  ]
                 ], device=device)[:]

const = Constants().to(device)

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2,0.0], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True
                   }


molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

# Set k for the dissipative electronic force term. See https://aip.scitation.org/doi/full/10.1063/1.3148075 for details.
output={'molid':[0, 1], 'thermo':1, 'dump':1, 'prefix':'Outputs/4_XL-BOMD_damp'}
xl_bomd_params={'k':6}

md =  XL_BOMD(xl_bomd_params=xl_bomd_params, damp=100.0,
              seqm_parameters=seqm_parameters, Temp=400.0, timestep=0.4, output=output).to(device)
md.initialize_velocity(molecule )
#remove center of mass velocity
# Info_log flag generates .txt wtih additional info for every step (orbital energies, dipole)
_ = md.run(molecule, 10, remove_com=[True, 1], Info_log=True)

Initialize velocities: zero_com
Doing initialization
Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   285.76   1.477473e-01 -1.350201e+00 -1.202454e+00 ||    380.21   1.965831e-01 1.468533e+00 1.665116e+00 || 
     2   252.81   1.307145e-01 -1.329620e+00 -1.198905e+00 ||    364.04   1.882235e-01 1.489924e+00 1.678148e+00 || 
     3   212.53   1.098847e-01 -1.302999e+00 -1.193114e+00 ||    362.30   1.873239e-01 1.496206e+00 1.683530e+00 || 
     4   171.70   8.877810e-02 -1.271570e+00 -1.182792e+00 ||    394.23   2.038348e-01 1.487865e+00 1.691700e+00 || 
     5   123.86   6.403865e-02 -1.240548e+00 -1.176510e+00 ||    458.25   2.369353e-01 1.464608e+00 1.701544e+00 || 
     6    71.23   3.683039e-02 -1.213981e+00 -1.177150e+00 ||    515.31   2.664349e-01 1.435996e+00 1.702431e+00 || 
     7    29.91   1.546423e-02 -1.197947e+00 -1.182483e+00 ||    573.15   2.963393e-01 1.405454e+00 1.701793e+00 || 
     8    13.90   7.185755e-03 -1.192610e+00 -1.185424e+00 ||    611.40  